In [1]:
%matplotlib inline
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from tensorflow.keras import backend
from keras import backend as K
import tensorflow as tf
import random
import collections
from keras.layers import Dropout
from keras.models import Sequential 
from keras.layers import Dense 
from pathlib import Path
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from six import StringIO
from copy import copy
from scipy import stats
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def encodeWithBLOSUM62(amino_acids):
    ## a function that returns the blosum62 vector given a certain aa 
    return list(BLOSUM62_MATRIX[amino_acids].values)

def blosumEncoding(data):
    ## a function that creates amino acid sequence that encode by blosum62 matrix 
    total_data_row =[]
    for each in data:
    
        eachrow =[]
        for aa in each:
            eachrow = eachrow+ encodeWithBLOSUM62(aa)
    
        total_data_row.append(eachrow)
    
    return pd.DataFrame(total_data_row)

def split(word): 
    return [char for char in word] 

COMMON_AMINO_ACIDS = collections.OrderedDict(sorted({
   "A": "Alanine",
   "R": "Arginine",
   "N": "Asparagine",
   "D": "Aspartic Acid",
   "C": "Cysteine",
   "E": "Glutamic Acid",
   "Q": "Glutamine",
   "G": "Glycine",
   "H": "Histidine",
   "I": "Isoleucine",
   "L": "Leucine",
   "K": "Lysine",
   "M": "Methionine",
   "F": "Phenylalanine",
   "P": "Proline",
   "S": "Serine",
   "T": "Threonine",
   "W": "Tryptophan",
   "Y": "Tyrosine",
   "V": "Valine",
   }.items()))
COMMON_AMINO_ACIDS_WITH_UNKNOWN = copy(COMMON_AMINO_ACIDS)
COMMON_AMINO_ACIDS_WITH_UNKNOWN["X"] = "Unknown"
   
AMINO_ACID_INDEX = dict((letter, i) for (i, letter) in enumerate(COMMON_AMINO_ACIDS_WITH_UNKNOWN))

AMINO_ACIDS = list(COMMON_AMINO_ACIDS_WITH_UNKNOWN.keys())
   
BLOSUM62_MATRIX = pd.read_csv(StringIO("""
  A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  X
   A  4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0  0
   R -1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3  0
   N -2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  0
   D -2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  0
   C  0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1  0
   Q -1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0
   E -1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  0
   G  0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3  0
   H -2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0
   I -1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3  0
   L -1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1  0
   K -1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2  0
   M -1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1  0
   F -2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1  0
   P -1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2  0
   S  1 -1  1  0 -1  0  0  0 -1 -2 -2  0 -1 -2 -1  4  1 -3 -2 -2  0
   T  0 -1  0 -1 -1 -1 -1 -2 -2 -1 -1 -1 -1 -2 -1  1  5 -2 -2  0  0 
   W -3 -3 -4 -4 -2 -2 -3 -2 -2 -3 -2 -3 -1  1 -4 -3 -2 11  2 -3  0
   Y -2 -2 -2 -3 -2 -1 -2 -3  2 -1 -1 -2 -1  3 -3 -2 -2  2  7 -1  0
   V  0 -3 -3 -3 -1 -2 -2 -3 -3  3  1 -2  1 -1 -2 -2  0 -3 -1  4  0
   X  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
   """), sep='\s+').loc[AMINO_ACIDS, AMINO_ACIDS].astype("int8")
assert (BLOSUM62_MATRIX == BLOSUM62_MATRIX.T).all().all() 


Using TensorFlow backend.


In [2]:
def makeSameLength(each_peptide):
    first = each_peptide
    middle = each_peptide.center(15,"X")
    end = ''
    
    while len(first) < 15:
        first +='X'
        end+='X' 
    end = end + each_peptide[-len(each_peptide):]
    return first+middle+end

In [89]:
## Data preprocessing, importing the dataset
## need to change the file location
workdir = Path('/Users/kivanc/DataMining-ML/Projects/FinalProject')
               #'/Volumes/Research/MAC_Research_Data/UGA/3Fall/DataMining6380/Mini_projecct_2/')
dataset = pd.read_csv(workdir/"HLA-A-01.txt",sep='\t')

peptides = dataset.peptide.values
final_peptide = [split(makeSameLength(each_peptide)) for each_peptide in peptides]

In [90]:
## encode y label
from sklearn import preprocessing
y = dataset.Binder.values
le = preprocessing.LabelEncoder()
a = le.fit(y)
y= le.transform(y)

## encode x label using bl62 matrix
X = blosumEncoding(final_peptide).values
## to see the dataset distribution 
dataset.groupby('Binder').size()
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split 
val_size = 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, shuffle= True,stratify = y)
X_train, X_val, y_train, y_val =  train_test_split(X_train, y_train, test_size=val_size, random_state = 0, shuffle=True)


In [91]:
## balance the class and create a new dataframe
from sklearn.utils import resample
Binder_max_count = dataset.groupby(['Binder']).count().max()[0]
Binders = dataset.Binder.unique()
Binders = np.delete(Binders, np.where(Binders == False), axis =0 )
df_majority = dataset[dataset.Binder==False]
for binder in Binders: 
    df_minority_upsampled = resample(dataset[dataset['Binder']==binder], 
                                 replace=True,     # sample with replacement
                                 n_samples=Binder_max_count,    # to match majority class
                                 random_state=42) # reproducible results
    
    df_majority = pd.concat([df_majority, df_minority_upsampled])

In [92]:
## Balance the classes
peptides = df_majority.peptide.values
final_peptide = [split(makeSameLength(each_peptide)) for each_peptide in peptides]

y = df_majority.Binder.values
le = preprocessing.LabelEncoder()
a = le.fit(y)
y= le.transform(y)

## encode x label using bl62 matrix
X = blosumEncoding(final_peptide).values
## to see the df_majority distribution 
df_majority.groupby('Binder').size()
# Splitting the df_majority into the Training set and Test set
from sklearn.model_selection import train_test_split 
val_size = 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, shuffle= True,stratify = y)
X_train, X_val, y_train, y_val =  train_test_split(X_train, y_train, test_size=val_size, random_state = 0, shuffle=True)

In [93]:
# Create an ensemble Neural Network in Keras

## Create an ANN model
# Training the model
# Dense object will take care to initialize the random number close to 0 ( first ANN step)
classifier = Sequential() # use the sequential layer
classifier.add(Dense(units = 80, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 945))
classifier.add(Dropout(rate = 0.5))
classifier.add(Dense(units = 80,  kernel_initializer = 'uniform', activation = 'tanh', input_dim = 945))
classifier.add(Dropout(rate = 0.5))
classifier.add(Dense(units = 80,   kernel_initializer = 'uniform', activation = 'tanh', input_dim = 945))
## here is the output layer
## if we deal with more than 2 categories, the activation function needs to use softmax
classifier.add(Dense(units = 2, kernel_initializer = 'uniform', activation = 'sigmoid'))
opt = keras.optimizers.rmsprop(learning_rate= 0.001)
# Compiling the ANN
classifier.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
# Fitting the ANN to the Training set
# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience= 50)
# check the model performance and save the best model
mc = ModelCheckpoint(
    'D:/GradSchool/Classes/DataMining/FinalProject_NN/Final_best_50_batch_model.h5', monitor='val_loss', mode='min', save_best_only=True)
    #'/Volumes/Research/MAC_Research_Data/UGA/3Fall/DataMining6380/Mini_projecct_2/Minproject2best_50batch_model.h5', monitor='val_loss', mode='min', save_best_only=True)

classifier.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_83 (Dense)             (None, 80)                75680     
_________________________________________________________________
dropout_41 (Dropout)         (None, 80)                0         
_________________________________________________________________
dense_84 (Dense)             (None, 80)                6480      
_________________________________________________________________
dropout_42 (Dropout)         (None, 80)                0         
_________________________________________________________________
dense_85 (Dense)             (None, 80)                6480      
_________________________________________________________________
dense_86 (Dense)             (None, 2)                 162       
Total params: 88,802
Trainable params: 88,802
Non-trainable params: 0
_________________________________________________

In [94]:
def fit_model(X_train, y_train):
    #define model
    classifier = Sequential() # use the sequential layer
    classifier.add(Dense(units = 80, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 945))
    classifier.add(Dropout(rate = 0.5))
    classifier.add(Dense(units = 80,  kernel_initializer = 'uniform', activation = 'tanh', input_dim = 945))
    classifier.add(Dropout(rate = 0.5))
    classifier.add(Dense(units = 80,   kernel_initializer = 'uniform', activation = 'tanh', input_dim = 945))
    ## here is the output layer
    ## if we deal with more than 2 categories, the activation function needs to use softmax
    
    #I changed the output layer to have 2 units
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    opt = keras.optimizers.rmsprop(learning_rate= 0.001)
    # Compiling the ANN
    classifier.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
    # Fitting the ANN to the Training set
    # simple early stopping
    classifier.fit(X_train, y_train, epochs=500, verbose=0)
    return classifier

In [95]:
from os import makedirs

# Delete the folder and start here if I want to do a clean run

# create directory for models
makedirs('models')

In [96]:
# fit and save models
n_members = 5
for i in range(n_members):
    # fit model
    model = fit_model(X_train, y_train)
    # save model
    filename = 'models/model_' + str(i + 1) + '.h5'
    model.save(filename)
    print('>Saved %s' % filename)

>Saved models/model_1.h5
>Saved models/model_2.h5
>Saved models/model_3.h5
>Saved models/model_4.h5
>Saved models/model_5.h5


In [97]:
# Start the separate stacking model

In [98]:
# load models from file
from keras.models import load_model

def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = 'models/model_' + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models


In [99]:
# load all models
n_members = 5
members = load_all_models(n_members)
print('Loaded %d models' % len(members))

>loaded models/model_1.h5
>loaded models/model_2.h5
>loaded models/model_3.h5
>loaded models/model_4.h5
>loaded models/model_5.h5
Loaded 5 models


In [100]:
# evaluate standalone models on test dataset
from keras.utils import to_categorical 

for model in members:
    #testy_enc = to_categorical(y_test)
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    print('Model Accuracy: %.3f' % acc)

Model Accuracy: 0.922
Model Accuracy: 0.922
Model Accuracy: 0.919
Model Accuracy: 0.919
Model Accuracy: 0.922


In [101]:
# create stacked model input dataset as outputs from the ensemble
import numpy as np
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        # make prediction
        yhat = model.predict(inputX, verbose=0)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = np.dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX


In [102]:
# fit a model based on the outputs from the ensemble members
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit standalone model
    #model = LogisticRegression()
    #model = DecisionTreeClassifier()
    #model = GaussianNB()
    model = RandomForestClassifier(n_estimators=100, random_state=18, max_features = 'sqrt',n_jobs=-1, verbose = 0)
    model.fit(stackedX, inputy)
    return model


In [103]:
# fit stacked model using the ensemble
model = fit_stacked_model(members, X_test, y_test)


In [104]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [105]:
from sklearn.metrics import accuracy_score
# evaluate model on test set
yhat = stacked_prediction(members, model, X_test)
acc = accuracy_score(y_test, yhat)
print('Stacked Test Accuracy: %.3f' % acc)


Stacked Test Accuracy: 0.994


In [ ]:
# Create two other classifiers with kfold cross validation

In [24]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

# xgboost
xgbscores = []
clf_xgb = XGBClassifier()
k_fold = StratifiedKFold(n_splits=10)
for train_indices, test_indices in k_fold.split(X,y):
    clf_xgb.fit(X[train_indices], y[train_indices]) 
    predicted_dt = clf_xgb.predict(X[test_indices])
    xgbscores.append(accuracy_score(y[test_indices], predicted_dt))
    
for i in xgbscores:
    print("each fold accuracy:", i)


each fold accuracy: 0.4794841735052755
each fold accuracy: 0.4912075029308324
each fold accuracy: 0.738569753810082
each fold accuracy: 0.7866354044548651
each fold accuracy: 0.8253223915592028
each fold accuracy: 0.7702227432590856
each fold accuracy: 0.8464243845252052
each fold accuracy: 0.8218053927315357
each fold accuracy: 0.784037558685446
each fold accuracy: 0.5199530516431925


In [25]:
# Ada Boost Classifier
from sklearn.ensemble import AdaBoostClassifier


abcscores = []
clf_abc = AdaBoostClassifier(n_estimators = 80, learning_rate = .001)
k_fold = StratifiedKFold(n_splits=10)
for train_indices, test_indices in k_fold.split(X,y):
    clf_abc.fit(X[train_indices], y[train_indices]) 
    predicted_dt = clf_abc.predict(X[test_indices])
    abcscores.append(accuracy_score(y[test_indices], predicted_dt))
    
for i in abcscores:
    print("each fold accuracy:", i)

each fold accuracy: 0.8147713950762017
each fold accuracy: 0.7713950762016413
each fold accuracy: 0.8053927315357562
each fold accuracy: 0.776084407971864
each fold accuracy: 0.7784290738569754
each fold accuracy: 0.8065650644783119
each fold accuracy: 0.8194607268464243
each fold accuracy: 0.794841735052755
each fold accuracy: 0.7946009389671361
each fold accuracy: 0.8204225352112676


In [35]:
# kfold cross validation with our NN
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
import numpy
from numpy import array
from numpy import argmax


# evaluate a single mlp model
def evaluate_model(trainX, trainy, testX, testy):
    # encode targets
    trainy_enc = to_categorical(trainy)
    testy_enc = to_categorical(testy)
    # define model
    model = Sequential() # use the sequential layer
    model.add(Dense(units = 80, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 945))
    model.add(Dropout(rate = 0.5))
    model.add(Dense(units = 80,  kernel_initializer = 'uniform', activation = 'tanh', input_dim = 945))
    model.add(Dropout(rate = 0.5))
    model.add(Dense(units = 80,   kernel_initializer = 'uniform', activation = 'tanh', input_dim = 945))
    ## here is the output layer
    ## if we deal with more than 2 categories, the activation function needs to use softmax
    
    #I changed the output layer to have 2 units
    model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    opt = keras.optimizers.rmsprop(learning_rate= 0.001)
    # Compiling the ANN
    model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
    # Fitting the ANN to the Training set
    # simple early stopping
    model.fit(X_train, y_train, epochs=500, verbose=0)
    _, test_acc = model.evaluate(testX, testy_enc, verbose=0)
    return model, test_acc
 
# make an ensemble prediction for multi-class classification
def ensemble_predictions(members, testX):
    # make predictions
    yhats = [model.predict(testX) for model in members]
    yhats = array(yhats)
    # sum across ensemble members
    summed = numpy.sum(yhats, axis=0)
    # argmax across classes
    result = argmax(summed, axis=1)
    return result
 
# evaluate a specific number of members in an ensemble
def evaluate_n_members(members, n_members, testX, testy):
    # select a subset of members
    subset = members[:n_members]
    # make prediction
    yhat = ensemble_predictions(subset, testX)
    # calculate accuracy
    return accuracy_score(testy, yhat)
 

# prepare the k-fold cross-validation configuration
n_folds = 10
kfold = KFold(n_folds, True, 1)
# cross validation estimation of performance
scores, members = list(), list()
for train_ix, test_ix in kfold.split(X):
    # select samples
    trainX, trainy = X[train_ix], y[train_ix]
    testX, testy = X[test_ix], y[test_ix]
    # evaluate model
    model, test_acc = evaluate_model(trainX, trainy, testX, testy)
    print('>%.3f' % test_acc)
    scores.append(test_acc)
    members.append(model)
# summarize expected performance
print('Estimated Accuracy %.3f (%.3f)' % (mean(scores), std(scores)))
# evaluate different numbers of ensembles on hold out set
single_scores, ensemble_scores = list(), list()
for i in range(1, n_folds+1):
    ensemble_score = evaluate_n_members(members, i, newX, newy)
    newy_enc = to_categorical(newy)
    _, single_score = members[i-1].evaluate(newX, newy_enc, verbose=0)
    print('> %d: single=%.3f, ensemble=%.3f' % (i, single_score, ensemble_score))
    ensemble_scores.append(ensemble_score)
    single_scores.append(single_score)
# plot score vs number of ensemble members
print('Accuracy %.3f (%.3f)' % (mean(single_scores), std(single_scores)))
x_axis = [i for i in range(1, n_folds+1)]
pyplot.plot(x_axis, single_scores, marker='o', linestyle='None')
pyplot.plot(x_axis, ensemble_scores, marker='o')
pyplot.show()


C:\Users\Corey\Anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


ValueError: Error when checking target: expected dense_30 to have shape (1,) but got array with shape (2,)

In [96]:
################################# This next step somehow hurts the accuracy?

In [97]:
## Start the Integrated Stacking Model

In [27]:
# update all layers in all models to not be trainable
for i in range(len(members)):
    model = members[i]
    for layer in model.layers:
        # make not trainable
        layer.trainable = False
        # rename to avoid 'unique layer name' issue
        layer._name = 'ensemble_' + str(i+1) + '_' + layer.name

In [28]:
from keras.layers.merge import concatenate
from keras.utils import plot_model
from keras.models import Model
from matplotlib import pyplot
# define stacked model from multiple member input models
def define_stacked_model(members):
    # update all layers in all models to not be trainable
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            # make not trainable
            layer.trainable = False
            # rename to avoid 'unique layer name' issue
            layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
    # define multi-headed input
    ensemble_visible = [model.input for model in members]
    # concatenate merge output from each model
    ensemble_outputs = [model.output for model in members]
    merge = concatenate(ensemble_outputs)
    hidden = Dense(units = 80, activation = 'tanh')(merge)
    output = Dense(units = 1, activation = 'sigmoid')(hidden)
    model = Model(inputs=ensemble_visible, outputs=output)
    # plot graph of ensemble
    plot_model(model, show_shapes=True, to_file='model_graph.png')
    # compile
    model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model


In [29]:
# define ensemble model

stacked_model = define_stacked_model(members)

In [30]:
# fit a stacked model
def fit_stacked_model(model, inputX, inputy):
    # prepare input data
    X = [inputX for _ in range(len(model.input))]
    # encode output data
    #inputy_enc = to_categorical(inputy)
    # fit model
    model.fit(X, inputy, epochs=200, verbose=0)

In [31]:
# fit stacked model on test dataset
fit_stacked_model(stacked_model, X_test, y_test)

In [32]:
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
    # prepare input data
    X = [inputX for _ in range(len(model.input))]
    # make prediction
    return model.predict(X, verbose=0)

In [34]:
from numpy import argmax
from numpy import array

# make predictions and evaluate
yhat = predict_stacked_model(stacked_model, X_test)
#yhat = argmax(yhat, axis=1)
yhat = array(yhat)
acc = accuracy_score(y_test, yhat)
print('Integrated Stacked Test Accuracy: %.3f' % acc)


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [90]:
yhat = stacked_prediction(members, model, X_test)
acc = accuracy_score(y_test, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

AttributeError: 'numpy.ndarray' object has no attribute 'head'